In [ ]:
# 1. Write a python program which searches all the product under a particular product from www.amazon.in.
# The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search
# for guitars.
# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your
# search results and save it in a data frame and csv. In case if any product has less than 3 pages in search
# results then scrape all the products available under that product name. Details to be scraped are: "Brand
# Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and
# “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“

# importing libraries
from bs4 import BeautifulSoup
import requests

def main(URL):
	# opening our output file in append mode
	File = open("out.csv", "a")

	# specifying user agent, You can use other user agents
	# available on the internet
	HEADERS = ({'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

	# Making the HTTP Request
	webpage = requests.get(URL, headers=HEADERS)

	# Creating the Soup Object containing all data
	soup = BeautifulSoup(webpage.content, "lxml")

	# retrieving product title
	try:
		# Outer Tag Object
		title = soup.find("span",
						attrs={"id": 'productTitle'})

		# Inner NavigableString Object
		title_value = title.string

		# Title as a string value
		title_string = title_value.strip().replace(',', '')

	except AttributeError:
		title_string = "NA"
	print("product Title = ", title_string)

	# saving the title in the file
	File.write(f"{title_string},")

	# retrieving price
	try:
		price = soup.find(
			"span", attrs={'id': 'priceblock_ourprice'})
								.string.strip().replace(',', '')
		# we are omitting unnecessary spaces
		# and commas form our string
	except AttributeError:
		price = "NA"
	print("Products price = ", price)

	# saving
	File.write(f"{price},")

	# retrieving product rating
	try:
		rating = soup.find("i", attrs={
						'class': 'a-icon a-icon-star a-star-4-5'})
									.string.strip().replace(',', '')

	except AttributeError:

		try:
			rating = soup.find(
				"span", attrs={'class': 'a-icon-alt'})
								.string.strip().replace(',', '')
		except:
			rating = "NA"
	print("Overall rating = ", rating)

	File.write(f"{rating},")

	try:
		review_count = soup.find(
			"span", attrs={'id': 'acrCustomerReviewText'})
								.string.strip().replace(',', '')

	except AttributeError:
		review_count = "NA"
	print("Total reviews = ", review_count)
	File.write(f"{review_count},")

	# print availablility status
	try:
		available = soup.find("div", attrs={'id': 'availability'})
		available = available.find("span")
					.string.strip().replace(',', '')

	except AttributeError:
		available = "NA"
	print("Availability = ", available)

	# saving the availability and closing the line
	File.write(f"{available},\n")

	# closing the file
	File.close()


if __name__ == '__main__':
# opening our url file to access URLs
	file = open("url.txt", "r")

	# iterating over the urls
	for links in file.readlines():
		main(links)


In [ ]:
# 3. Write a python program to access the search bar and search button on images.google.com and scrape 10
# images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

import os
import time

import io
import hashlib
import signal
from glob import glob
import requests

from PIL import Image
from selenium import webdriver

number_of_images = 400
GET_IMAGE_TIMEOUT = 2
SLEEP_BETWEEN_INTERACTIONS = 0.1
SLEEP_BEFORE_MORE = 5
IMAGE_QUALITY = 85

output_path = "/path/to/your/image/directory"

search_terms = [
    "fruits",
    "cars",
    "Machine Learning",
    "Guitar",
    "Cakes",
]

dirs = glob(output_path + "*")
dirs = [dir.split("/")[-1].replace("_", " ") for dir in dirs]
search_terms = [term for term in search_terms if term not in dirs]

wd = webdriver.Chrome()
wd.get("https://google.com")

class timeout:
    def __init__(self, seconds=1, error_message="Timeout"):
        self.seconds = seconds
        self.error_message = error_message

    def handle_timeout(self, signum, frame):
        raise TimeoutError(self.error_message)

    def __enter__(self):
        signal.signal(signal.SIGALRM, self.handle_timeout)
        signal.alarm(self.seconds)

    def __exit__(self, type, value, traceback):
        signal.alarm(0)
        
    def fetch_image_urls(
    query: str,
    max_links_to_fetch: int,
    wd: webdriver,
    sleep_between_interactions: int = 1,
):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

    # Build the Google Query.
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    # Declared as a set, to prevent duplicates.
    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # Get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)

        print(
            f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}"
        )

        # Loop through image thumbnail identified
        for img in thumbnail_results[results_start:number_results]:
            # Try to click every thumbnail such that we can get the real image behind it.
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # Extract image urls
            actual_images = wd.find_elements_by_css_selector("img.n3VNCb")
            for actual_image in actual_images:
                if actual_image.get_attribute(
                    "src"
                ) and "http" in actual_image.get_attribute("src"):
                    image_urls.add(actual_image.get_attribute("src"))

            image_count = len(image_urls)

            # If the number images found exceeds our `num_of_images`, end the seaerch.
            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            # If we haven't found all the images we want, let's look for more.
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(SLEEP_BEFORE_MORE)

            # Check for button signifying no more images.
            not_what_you_want_button = ""
            try:
                not_what_you_want_button = wd.find_element_by_css_selector(".r0zKGf")
            except:
                pass

            # If there are no more images return.
            if not_what_you_want_button:
                print("No more images available.")
                return image_urls

            # If there is a "Load More" button, click it.
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button and not not_what_you_want_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # Move the result startpoint further down.
        results_start = len(thumbnail_results)

    return image_urls


def persist_image(folder_path: str, url: str):
    try:
        print("Getting image")
        # Download the image.  If timeout is exceeded, throw an error.
        with timeout(GET_IMAGE_TIMEOUT):
            image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        # Convert the image into a bit stream, then save it.
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert("RGB")
        # Create a unique filepath from the contents of the image.
        file_path = os.path.join(
            folder_path, hashlib.sha1(image_content).hexdigest()[:10] + ".jpg"
        )
        with open(file_path, "wb") as f:
            image.save(f, "JPEG", quality=IMAGE_QUALITY)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

def search_and_download(search_term: str, target_path="./images", number_images=5):
    # Create a folder name.
    target_folder = os.path.join(target_path, "_".join(search_term.lower().split(" ")))

    # Create image folder if needed.
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    # Open Chrome
    with webdriver.Chrome() as wd:
        # Search for images URLs.
        res = fetch_image_urls(
            search_term,
            number_images,
            wd=wd,
            sleep_between_interactions=SLEEP_BETWEEN_INTERACTIONS,
        )

        # Download the images.
        if res is not None:
            for elem in res:
                persist_image(target_folder, elem)
        else:
            print(f"Failed to return links for term: {search_term}")

# Loop through all the search terms.
for term in search_terms:
    search_and_download(term, output_path, number_of_images)

In [ ]:
# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.


from selenium import webdriver
from BeautifulSoup import BeautifulSoup
import pandas as pd

driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")

products=[] #List to store name of the product
prices=[] #List to store price of the product
ratings=[] #List to store rating of the product
driver.get("https://www.flipkart.com/search?q=pixel+4a&sid=tyy%2C4io&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_1_5_na_na_na&otracker1=AS_QueryStore_OrganicAutoSuggest_1_5_na_na_na&as-pos=1&as-type=HISTORY&suggestionId=pixel+4a%7CMobiles&requestId=737ec53e-7da5-424c-9eea-815cd8befbe1")
content = driver.page_source
soup = BeautifulSoup(content)
for a in soup.findAll('a',href=True, attrs={'class':'_31qSD5'}):
name=a.find('div', attrs={'class':'_3wU53n'})
price=a.find('div', attrs={'class':'_1vC4OE _2rQ-NK'})
rating=a.find('div', attrs={'class':'hGSR34 _2beYZw'})
products.append(name.text)
prices.append(price.text)
ratings.append(rating.text)

python web-s.py
df = pd.DataFrame({'Product Name':products,'Price':prices,'Rating':ratings})df.to_csv('products.csv', index=False, encoding='utf-8')

In [ ]:
# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

# import required modules
from selenium import webdriver
from time import sleep


# assign url in the webdriver object
driver = webdriver.Chrome()
driver.get("https://www.google.co.in/maps/@10.8091781,78.2885026,7z")
sleep(2)


# search locations
def searchplace():
	Place = driver.find_element_by_class_name("tactile-searchbox-input")
	Place.send_keys("Tiruchirappalli")
	Submit = driver.find_element_by_xpath(
		"/html/body/jsl/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/div[1]/button")
	Submit.click()

searchplace()


# get directions
def directions():
	sleep(10)
	directions = driver.find_element_by_xpath(
		"/html/body/jsl/div[3]/div[9]/div[7]/div/div[1]/div/div/div[5]/div[1]/div/button")
	directions.click()

directions()


# find place
def find():
	sleep(6)
	find = driver.find_element_by_xpath(
		"/html/body/jsl/div[3]/div[9]/div[3]/div[1]/div[2]/div/div[3]/div[1]/div[1]/div[2]/div/div/input")
	find.send_keys("Tirunelveli")
	sleep(2)
	search = driver.find_element_by_xpath(
		"/html/body/jsl/div[3]/div[9]/div[3]/div[1]/div[2]/div/div[3]/div[1]/div[1]/div[2]/button[1]")
	search.click()

find()


# get transportation details
def kilometers():
	sleep(5)
	Totalkilometers = driver.find_element_by_xpath(
		"/html/body/jsl/div[3]/div[9]/div[7]/div/div[1]/div/div/div[5]/div[1]/div[1]/div[1]/div[1]/div[2]/div")
	print("Total Kilometers:", Totalkilometers.text)
	sleep(5)
	Bus = driver.find_element_by_xpath(
		"/html/body/jsl/div[3]/div[9]/div[7]/div/div[1]/div/div/div[5]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]")
	print("Bus Travel:", Bus.text)
	sleep(7)
	Train = driver.find_element_by_xpath(
		"/html/body/jsl/div[3]/div[9]/div[7]/div/div[1]/div/div/div[5]/div[2]/div[1]/div[2]/div[1]/div")
	print("Train Travel:", Train.text)
	sleep(7)

kilometers()


In [ ]:
# 7. Write a program to scrap all the available details of best gaming laptops from digit.in.

from selenium import webdriver
from BeautifulSoup import BeautifulSoup
import pandas as pd

driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")

Operating Systems=[] #List to store Details of the operating system
Displays=[] #List to store the details of the Display
Processors=[] #List to store the details of processor
driver.get("https://www.digit.in/top-products/best-gaming-laptops-40.html")
content = driver.page_source
soup = BeautifulSoup(content)
for a in soup.findAll('a',href=True, attrs={'class':'_31qSD5'}):
OS=a.find('div', attrs={'class':'_3wU53n'})
Display=a.find('div', attrs={'class':'_1vC4OE _2rQ-NK'})
Processor=a.find('div', attrs={'class':'hGSR34 _2beYZw'})
Operating Systems.append(name.text)
Displays.append(price.text)
Processors.append(rating.text)

python web-s.py
df = pd.DataFrame({'OS Name':Operating Systems,'Display':Displays,'Processor':Processors})df.to_csv('products.csv', index=False, encoding='utf-8')

In [ ]:
# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

import numpy as np
import pandas as pd
import pandas.util
forbes = pd.read_csv('reviews_static.csv', names = ['Rank', 'Name', 'Net Worth', 'Age', 'Source', 'Country'])

forbes_realtime = pd.read_csv('reviews_realtime.csv', names = ['Rank', 'Name', 'Net Worth', 'Age', 'Source', 'Country'])

forbes_women = pd.read_csv('reviews_static_women.csv', names = ['Rank', 'Name', 'Net Worth', 'Age', 'Source', 'Country'])
#creating data frame for current net worth
realtime = forbes_realtime[['Name','Net Worth']]

#merging dataframe
forbes_data = pd.merge(forbes, realtime,how = 'left', on = 'Name')

#cleaning NA values
forbes_data['Age'] = forbes_data['Age'].fillna(forbes_data['Age'].median())
forbes_data['Net Worth_y'] = forbes_data['Net Worth_y'].fillna(0)

#creating new column "Change"
forbes_data['Change'] = forbes_data['Net Worth_y']- forbes_data['Net Worth_x']
# changing float into integar
forbes_data['Age'] = list(map(lambda x: int(x), forbes_data['Age']))
forbes_data['Rank'] = list(map(lambda x: int(x), forbes_data['Rank']))
#creating women data frame, so i can merge that into main data frame to create Gender column
women = forbes_women['Name']

#Creating new column "Gender" in main data frame
forbes_data['Gender'] = forbes_data['Name'].isin(women)
forbes_data['Gender'].replace([True,False],['F','M'],inplace = True)
#checking final data frame
forbes_data.head(10)

In [ ]:
# 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

import requests
import json
import time


def search_dict(partial, key):
    """
    A handy function that searches for a specific `key` in a `data` dictionary/list
    """
    if isinstance(partial, dict):
        for k, v in partial.items():
            if k == key:
                # found the key, return the value
                yield v
            else:
                # value of the dict may be another dict, so we search there again
                for o in search_dict(v, key):
                    yield o
    elif isinstance(partial, list):
        # if the passed data is a list
        # iterate over it & search for the key at the items in the list
        for i in partial:
            for o in search_dict(i, key):
                yield o


def find_value(html, key, num_sep_chars=2, separator='"'):
    # define the start position by the position of the key + 
    # length of key + separator length (usually : and ")
    start_pos = html.find(key) + len(key) + num_sep_chars
    # the end position is the position of the separator (such as ")
    # starting from the start_pos
    end_pos = html.find(separator, start_pos)
    # return the content in this range
    return html[start_pos:end_pos]


def get_comments(url):
    session = requests.Session()
    # make the request
    res = session.get(url)
    # extract the XSRF token
    xsrf_token = find_value(res.text, "XSRF_TOKEN", num_sep_chars=3)
    # parse the YouTube initial data in the <script> tag
    data_str = find_value(res.text, 'window["ytInitialData"] = ', num_sep_chars=0, separator="\n").rstrip(";")
    # convert to Python dictionary instead of plain text string
    data = json.loads(data_str)
    # search for the ctoken & continuation parameter fields
    for r in search_dict(data, "itemSectionRenderer"):
        pagination_data = next(search_dict(r, "nextContinuationData"))
        if pagination_data:
            # if we got something, break out of the loop,
            # we have the data we need
            break

    continuation_tokens = [(pagination_data['continuation'], pagination_data['clickTrackingParams'])]

    while continuation_tokens:
        # keep looping until continuation tokens list is empty (no more comments)
        continuation, itct = continuation_tokens.pop()
    
        # construct params parameter (the ones in the URL)
        params = {
            "action_get_comments": 1,
            "pbj": 1,
            "ctoken": continuation,
            "continuation": continuation,
            "itct": itct,
        }

        # construct POST body data, which consists of the XSRF token
        data = {
            "session_token": xsrf_token,
        }

        # construct request headers
        headers = {
            "x-youtube-client-name": "1",
            "x-youtube-client-version": "2.20200731.02.01"
        }

        # make the POST request to get the comments data
        response = session.post("https://www.youtube.com/comment_service_ajax", params=params, data=data, headers=headers)
        # convert to a Python dictionary
        comments_data = json.loads(response.text)

        for comment in search_dict(comments_data, "commentRenderer"):
            # iterate over loaded comments and yield useful info
            yield {
                "commentId": comment["commentId"],
                "text": ''.join([c['text'] for c in comment['contentText']['runs']]),
                "time": comment['publishedTimeText']['runs'][0]['text'],
                "isLiked": comment["isLiked"],
                "likeCount": comment["likeCount"],
                # "replyCount": comment["replyCount"],
                'author': comment.get('authorText', {}).get('simpleText', ''),
                'channel': comment['authorEndpoint']['browseEndpoint']['browseId'],
                'votes': comment.get('voteCount', {}).get('simpleText', '0'),
                'photo': comment['authorThumbnail']['thumbnails'][-1]['url'],
                "authorIsChannelOwner": comment["authorIsChannelOwner"],
            }

        # load continuation tokens for next comments (ctoken & itct)
        continuation_tokens = [(next_cdata['continuation'], next_cdata['clickTrackingParams'])
                         for next_cdata in search_dict(comments_data, 'nextContinuationData')] + continuation_tokens

        # avoid heavy loads with popular videos
        time.sleep(0.1)
    




if __name__ == "__main__":
    # from pprint import pprint
    # url = "https://www.youtube.com/watch?v=jNQXAC9IVRw"
    # for count, comment in enumerate(get_comments(url)):
    #     if count == 3:
    #         break
    #     pprint(comment)
    #     print("="*50)
    import argparse
    import os

    parser = argparse.ArgumentParser(description="Simple YouTube Comment extractor")
    parser.add_argument("url", help="The YouTube video full URL")
    parser.add_argument("-l", "--limit", type=int, help="Number of maximum comments to extract, helpful for longer videos")
    parser.add_argument("-o", "--output", help="Output JSON file, e.g data.json")

    # parse passed arguments
    args = parser.parse_args()
    limit = args.limit
    output = args.output
    url = args.url

    from pprint import pprint
    for count, comment in enumerate(get_comments(url)):
        if limit and count >= limit:
            # break out of the loop when we exceed limit specified
            break
        if output:
            # write comment as JSON to a file
            with open(output, "a") as f:
                # begin writing, adding an opening brackets
                if count == 0:
                    f.write("[")
                f.write(json.dumps(comment, ensure_ascii=False) + ",")
        else:
            pprint(comment)
            print("="*50)
    print("total comments extracted:", count)
    if output:
        # remove the last comma ','
        with open(output, "rb+") as f:
            f.seek(-1, os.SEEK_END)
            f.truncate()
        # add "]" to close the list in the end of the file
        with open(output, "a") as f:
            print("]", file=f)

In [ ]:
# 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location.

import selenium
import selenium.webdriver
import time
import random
import os
from bs4 import BeautifulSoup

data = []

final_list = [
    'https://www.hostelworld.com/hostels/London'
]

# load your driver only once to save time
driver = selenium.webdriver.Chrome()

for url in final_list:
    data.append({})

    # cache the HTML code to the filesystem
    # generate a filename from the URL where all non-alphanumeric characters (e.g. :/) are replaced with underscores _
    filename = ''.join([s if s.isalnum() else '_' for s in url])
    if not os.path.isfile(filename):
        driver.get(url)
        
        # better use selenium's wait functions here  
        time.sleep(random.randint(10, 20))
        source = driver.page_source
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(source)
    else:
        with open(filename, 'r', encoding='utf-8') as f:
            source = f.read()
    soup = BeautifulSoup(source, 'html.parser')

    review = soup.find_all(class_='reviews')[-1]
    
    try:
        price = soup.find_all('span', attrs={'class':'price'})[-1] 
    except:
        price = soup.find_all('span', attrs={'class':'price'})

    data[-1]['name'] = soup.find_all(class_=['title-2'])[0].text.strip()
    
    rating_labels = soup.find_all(class_=['rating-label body-3'])
    rating_scores = soup.find_all(class_=['rating-score body-3'])
    assert len(rating_labels) == len(rating_scores)
    for label, score in zip(rating_labels, rating_scores):
        data[-1][label.text.strip()] = score.text.strip()
    
    data[-1]['price'] = price.text.strip()
    data[-1]['review'] = review.text.strip()